In [1]:
!pip install torch torchvision torchaudio

In [25]:
from sample import BaseModel, train_predictor, train_selector
from sample import PredictorNetwork
from sample import SelectorNetwork
import torchvision.models as models
import torch
import torch.nn as nn
import torch.optim as optim
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load the pretrained ResNet-50 model
resnet50 = models.resnet50(pretrained=True)
selector = models.convnext_tiny(pretrained=True)
predictor = models.convnext_small(pretrained=True)
# Switch to evaluation mode
resnet50.eval()
selector.eval()
predictor.eval()

# Define your input data (example input tensor)
input_data = torch.randn(1, 3, 224, 224)  # Example input with shape (batch_size, channels, height, width)

# Iterate through each layer in the ResNet-50 model and apply them sequentially
output = input_data
total_start = time.time()
for name, layer in resnet50.named_children():
    start = time.time()
    if name == 'avgpool':
        output = nn.functional.adaptive_avg_pool2d(output, (1, 1))
    elif name == 'fc':
        output = output.view(output.size(0), -1)
    elif name == 'layer1' or name == 'layer2' or name == 'layer3' or name == 'layer4':
        pred_out = predictor(output)
        if selector(pred_out) == 1:
            output = pred_out
            break

    print(f"Layer: {name}, Output shape: {output.shape}, total time: {time.time() - start}")

print(f"total time: {time.time() - total_start}")
#mod = models.convnext_tiny(pretrained=False)
#print(len(mod.named_children()))


cpu


c:\Users\cbfar\anaconda3\envs\cs221\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\cbfar\anaconda3\envs\cs221\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\cbfar\anaconda3\envs\cs221\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing 

Layer: conv1, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: bn1, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: relu, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: maxpool, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: layer1, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: layer2, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: layer3, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: layer4, Output shape: torch.Size([1, 3, 224, 224]), total time: 0.0
Layer: avgpool, Output shape: torch.Size([1, 3, 1, 1]), total time: 0.0019979476928710938
Layer: fc, Output shape: torch.Size([1, 3]), total time: 0.0
total time: 0.0030007362365722656
